# 第一階段：數據探索與理解
## 步驟 1：載入與檢查數據

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import os

# 創建專案目錄結構
project_dir = "semiconductor_maintenance"
os.makedirs(project_dir, exist_ok=True)
os.makedirs(f"{project_dir}/data", exist_ok=True)
os.makedirs(f"{project_dir}/models", exist_ok=True)
os.makedirs(f"{project_dir}/results", exist_ok=True)
os.makedirs(f"{project_dir}/visualizations", exist_ok=True)

# 載入數據
X = pd.read_csv('secom.data', delimiter=' ', header=None)
labels_df = pd.read_csv('secom_labels.data', delimiter=' ', header=None)

# 分離標籤和時間戳
y = labels_df[0]  # 第一列是標籤 (-1 = 故障, 1 = 正常)
timestamps = labels_df[1] if labels_df.shape[1] > 1 else None

# 將數據和標籤合併為一個DataFrame方便分析
if 'secom.names' in os.listdir():
    with open('secom.names', 'r') as f:
        feature_names = [line.strip() for line in f if line.strip()]
    if len(feature_names) == X.shape[1]:
        X.columns = feature_names
else:
    X.columns = [f'Feature_{i}' for i in range(X.shape[1])]

# 基本數據資料
print(f"數據形狀: {X.shape}")
print(f"總樣本數: {len(y)}")
print(f"故障樣本數: {sum(y == -1)}")
print(f"正常樣本數: {sum(y == 1)}")
print(f"故障率: {sum(y == -1) / len(y):.2%}")

# 檢查缺失值
missing_values = X.isnull().sum()
print(f"\n缺失值總數: {missing_values.sum()}")
print(f"有缺失值的特徵數: {sum(missing_values > 0)}")
print(f"缺失值最多的5個特徵:\n{missing_values.sort_values(ascending=False).head()}")

# 保存基本資料到文件
with open(f"{project_dir}/results/data_summary.txt", 'w') as f:
    f.write(f"數據形狀: {X.shape}\n")
    f.write(f"總樣本數: {len(y)}\n")
    f.write(f"故障樣本數: {sum(y == -1)}\n")
    f.write(f"正常樣本數: {sum(y == 1)}\n")
    f.write(f"故障率: {sum(y == -1) / len(y):.2%}\n")
    f.write(f"\n缺失值總數: {missing_values.sum()}\n")
    f.write(f"有缺失值的特徵數: {sum(missing_values > 0)}\n")

數據形狀: (1567, 590)
總樣本數: 1567
故障樣本數: 1463
正常樣本數: 104
故障率: 93.36%

缺失值總數: 41951
有缺失值的特徵數: 538
缺失值最多的5個特徵:
157    1429
292    1429
293    1429
158    1429
492    1341
dtype: int64


## 步驟 2：數據可視化探索

In [5]:
# 基本統計可視化
plt.figure(figsize=(12, 6), facecolor='white')
y_value_counts = pd.Series(y).value_counts()
ax = sns.barplot(x=y_value_counts.index, y=y_value_counts.values, palette='pastel')
ax.set_facecolor('white')
plt.title('Class Distribution')
plt.xlabel('Class (-1: Failure, 1: Normal)')
plt.ylabel('Sample Count')
plt.xticks(ticks=[0, 1], labels=['Failure', 'Normal'])
plt.grid(False)
plt.savefig(f"{project_dir}/visualizations/class_distribution.png", bbox_inches='tight', facecolor='white')
plt.close()

# 缺失值熱圖 (針對前100個特徵)
plt.figure(figsize=(20, 10), facecolor='white')
ax = sns.heatmap(X.iloc[:, :100].isnull(), cbar=False, yticklabels=False)
ax.set_facecolor('white')
plt.title('Missing Values Distribution (First 100 Features)')
plt.savefig(f"{project_dir}/visualizations/missing_values_heatmap.png", bbox_inches='tight', facecolor='white')
plt.close()

# 檢查特徵分布 (抽樣10個特徵)
sampled_features = X.columns[np.random.choice(X.shape[1], 10, replace=False)]
plt.figure(figsize=(15, 10), facecolor='white')
for i, feature in enumerate(sampled_features, 1):
    plt.subplot(2, 5, i)
    ax = sns.histplot(X[feature].dropna(), kde=True)
    ax.set_facecolor('white')
    plt.title(f'Feature: {feature}')
plt.tight_layout()
plt.savefig(f"{project_dir}/visualizations/feature_distributions.png", bbox_inches='tight', facecolor='white')
plt.close()

# 檢查特徵間的相關性 (抽樣30個特徵)
sampled_features = X.columns[np.random.choice(X.shape[1], 30, replace=False)]
correlation_matrix = X[sampled_features].corr()
plt.figure(figsize=(16, 14), facecolor='white')
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
ax = sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_facecolor('white')
plt.title('Feature Correlation (30 Sampled Features)')
plt.savefig(f"{project_dir}/visualizations/correlation_heatmap.png", bbox_inches='tight', facecolor='white')
plt.close()

# 箱型圖分析 - 比較正常與故障樣本在幾個主要特徵上的差異
X_with_labels = X.copy()
X_with_labels['label'] = y
important_features = []

# 找出與標籤相關性較高的特徵
for col in X.columns:
    if X[col].isnull().sum() < len(X) * 0.5:  # 只看缺失值少於50%的特徵
        correlation = X[col].fillna(X[col].median()).corr(pd.Series(y))
        if abs(correlation) > 0.1:  # 相關係數絕對值大於0.1
            important_features.append((col, correlation))

important_features = sorted(important_features, key=lambda x: abs(x[1]), reverse=True)[:10]
important_feature_names = [f[0] for f in important_features]

plt.figure(figsize=(15, 10), facecolor='white')
for i, feature in enumerate(important_feature_names[:5], 1):
    plt.subplot(1, 5, i)
    ax = sns.boxplot(x='label', y=feature, data=X_with_labels)
    ax.set_facecolor('white')
    plt.title(f'{feature} vs Label')
    plt.xlabel('Class (-1: Failure, 1: Normal)')
plt.tight_layout()
plt.savefig(f"{project_dir}/visualizations/boxplots_by_class.png", bbox_inches='tight', facecolor='white')
plt.close()

C:\Users\User\AppData\Local\Temp\ipykernel_9756\560604038.py:4: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x=y_value_counts.index, y=y_value_counts.values, palette='pastel')
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\function_base.py:2897: Runt

# 第二階段：數據預處理
## 步驟 3：處理缺失值和異常值

In [6]:
# 查看各特徵的缺失率
missing_percentages = (X.isnull().sum() / len(X)) * 100
high_missing_features = missing_percentages[missing_percentages > 30].index.tolist()

print(f"缺失率超過30%的特徵數: {len(high_missing_features)}")

# 移除缺失率高的特徵
X_filtered = X.drop(columns=high_missing_features)
print(f"過濾後的特徵數: {X_filtered.shape[1]}")

# 填補剩餘缺失值
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X_filtered), columns=X_filtered.columns)

# 保存預處理結果
X_imputed['label'] = y
X_imputed.to_csv(f"{project_dir}/data/preprocessed_data.csv", index=False)
print(f"預處理後的數據保存至 {project_dir}/data/preprocessed_data.csv")

# 驗證數據預處理效果
print(f"預處理前的缺失值數: {X.isnull().sum().sum()}")
print(f"預處理後的缺失值數: {X_imputed.isnull().sum().sum()}")

缺失率超過30%的特徵數: 32
過濾後的特徵數: 558
預處理後的數據保存至 semiconductor_maintenance/data/preprocessed_data.csv
預處理前的缺失值數: 41951
預處理後的缺失值數: 0


## 步驟 4：特徵工程與選擇

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 從預處理後的數據中分離特徵和標籤
data = pd.read_csv(f"{project_dir}/data/preprocessed_data.csv")
y = data['label']
X = data.drop('label', axis=1)

# 標準化數據
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 保存數據描述統計
X_stats = X.describe().T
# 重新計算缺失值百分比，而不是使用先前的missing_percentages變數
X_stats['missing_percentage'] = X.isnull().mean() * 100
X_stats.to_csv(f"{project_dir}/results/feature_statistics.csv")

# 方法1: 單變量特徵選擇 (ANOVA F-value)
selector_f = SelectKBest(f_classif, k=50)
X_selected_f = selector_f.fit_transform(X_scaled, y)
selected_feature_indices_f = selector_f.get_support(indices=True)
selected_features_f = X.columns[selected_feature_indices_f].tolist()

# 方法2: 相互資訊 (Mutual information) 特徵選擇
selector_mi = SelectKBest(mutual_info_classif, k=50)
X_selected_mi = selector_mi.fit_transform(X_scaled, y)
selected_feature_indices_mi = selector_mi.get_support(indices=True)
selected_features_mi = X.columns[selected_feature_indices_mi].tolist()

# 方法3: PCA降維
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_scaled)
explained_variance = pca.explained_variance_ratio_

# 可視化 PCA 方差解釋率
plt.figure(figsize=(10, 6), facecolor='white')  
plt.plot(np.cumsum(explained_variance))
plt.xlabel('Number of PCA Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance by PCA Components')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% Variance')
plt.grid(True)
plt.legend()
plt.savefig(f"{project_dir}/visualizations/pca_explained_variance.png")
plt.close()

# 保存特徵選擇結果
pd.DataFrame({
    'ANOVA_F_Selected_Features': pd.Series(selected_features_f),
    'Mutual_Info_Selected_Features': pd.Series(selected_features_mi)
}).to_csv(f"{project_dir}/results/selected_features.csv", index=False)

# 準備不同特徵集的訓練數據
feature_sets = {
    'original': X_scaled,
    'anova_f': X_scaled[selected_features_f],
    'mutual_info': X_scaled[selected_features_mi],
    'pca': pd.DataFrame(X_pca)
}

for name, dataset in feature_sets.items():
    dataset_with_label = pd.DataFrame(dataset)
    dataset_with_label['label'] = y
    dataset_with_label.to_csv(f"{project_dir}/data/feature_set_{name}.csv", index=False)
    print(f"特徵集 '{name}' 保存於 {project_dir}/data/feature_set_{name}.csv")

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  5  13  42  49  52  69  94 134 142 169 170 177 180 181 182 183 184 185
 216 219 220 221 222 223 224 225 226 227 230 231 232 233 242 243 244 245
 246 247 248 249 250 251 252 262 270 297 298 299 306 309 310 311 312 313
 314 345 350 351 352 353 354 355 356 359 360 361 362 371 372 373 374 375
 376 377 378 379 380 381 391 399 426 427 428 435 438 439 440 441 442 443
 458 474 477 478 479 480 481 482 483 484 485 488 489 490 491 500 501 502
 503 504 505 506 507 508 509 510] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invali

特徵集 'original' 保存於 semiconductor_maintenance/data/feature_set_original.csv
特徵集 'anova_f' 保存於 semiconductor_maintenance/data/feature_set_anova_f.csv
特徵集 'mutual_info' 保存於 semiconductor_maintenance/data/feature_set_mutual_info.csv
特徵集 'pca' 保存於 semiconductor_maintenance/data/feature_set_pca.csv


# 第三階段：模型開發與評估
## 步驟 5：建立基準模型

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
import joblib

# 載入處理後的數據集
feature_set_name = 'anova_f'  # 可選: original, anova_f, mutual_info, pca
data = pd.read_csv(f"{project_dir}/data/feature_set_{feature_set_name}.csv")
y = data['label']
X = data.drop('label', axis=1)

# 處理類別不平衡
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 資料分割 - 訓練、驗證與測試集
X_temp, X_test, y_temp, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)  # 0.25 x 0.8 = 0.2

print(f"訓練集: {X_train.shape[0]} 樣本")
print(f"驗證集: {X_val.shape[0]} 樣本")
print(f"測試集: {X_test.shape[0]} 樣本")

# 建立不同模型
models = {
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42),
    'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'gradient_boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'svm': SVC(probability=True, random_state=42),
    'mlp': MLPClassifier(hidden_layer_sizes=(100,50), max_iter=1000, random_state=42)
}

# 模型評估結果
results = {}

for name, model in models.items():
    # 訓練模型
    model.fit(X_train, y_train)
    
    # 預測驗證集
    y_val_pred = model.predict(X_val)
    
    # 計算評估指標
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, pos_label=-1)  # 故障為正例
    recall = recall_score(y_val, y_val_pred, pos_label=-1)
    f1 = f1_score(y_val, y_val_pred, pos_label=-1)
    
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    # 混淆矩陣 (Confusion Matrix)
    cm = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(8, 6), facecolor='white')  # 設定背景為白色
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    plt.savefig(f"{project_dir}/visualizations/confusion_matrix_{name}.png")
    plt.close()

    
    # ROC 曲線
    if hasattr(model, "predict_proba"):
        y_val_score = model.predict_proba(X_val)[:,1] if len(model.classes_) == 2 else model.predict_proba(X_val)[:, model.classes_ == -1].ravel()
    else:  # SVC with probability=False
        y_val_score = model.decision_function(X_val)
        
    fpr, tpr, _ = roc_curve(y_val == -1, y_val_score)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6), facecolor='white')
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {name}')
    plt.legend(loc='lower right')
    plt.savefig(f"{project_dir}/visualizations/roc_curve_{name}.png")
    plt.close()
    
    # 保存模型
    joblib.dump(model, f"{project_dir}/models/{name}_model.pkl")

# 保存模型評估結果
results_df = pd.DataFrame(results).T
results_df.to_csv(f"{project_dir}/results/model_evaluation.csv")
print(results_df)

# 找出最佳模型
best_model_name = results_df['f1_score'].idxmax()
print(f"最佳模型是: {best_model_name}，F1分數: {results_df.loc[best_model_name, 'f1_score']:.4f}")

訓練集: 1755 樣本
驗證集: 585 樣本
測試集: 586 樣本
                     accuracy  precision    recall  f1_score
logistic_regression  0.813675   0.845865  0.767918  0.805009
random_forest        0.957265   0.962069  0.952218  0.957118
gradient_boosting    0.924786   0.943060  0.904437  0.923345
svm                  0.878632   0.933594  0.815700  0.870674
mlp                  0.964103   1.000000  0.928328  0.962832
最佳模型是: mlp，F1分數: 0.9628


## 步驟 6：模型優化與調參

In [9]:
from sklearn.model_selection import GridSearchCV

# 選擇最佳模型類型進行優化
best_model_type = best_model_name

# 為不同模型設定參數網格
param_grids = {
    'logistic_regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    'random_forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'gradient_boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5]
    },
    'svm': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1],
        'kernel': ['rbf', 'linear']
    },
    'mlp': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    }
}

# 使用GridSearchCV進行參數優化
print(f"為模型 '{best_model_type}' 進行參數優化...")
model = models[best_model_type]
param_grid = param_grids[best_model_type]

# 使用StratifiedKFold確保每個折疊中類別分布一致
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    model, param_grid, cv=cv, scoring='f1', 
    verbose=1, n_jobs=-1, return_train_score=True
)

grid_search.fit(X_train, y_train)

print(f"最佳參數: {grid_search.best_params_}")
print(f"最佳交叉驗證F1分數: {grid_search.best_score_:.4f}")

# 儲存最佳模型
best_model = grid_search.best_estimator_
joblib.dump(best_model, f"{project_dir}/models/optimized_{best_model_type}_model.pkl")

# 使用最佳模型進行預測
y_test_pred = best_model.predict(X_test)

# 計算測試集性能指標
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, pos_label=-1)
test_recall = recall_score(y_test, y_test_pred, pos_label=-1)
test_f1 = f1_score(y_test, y_test_pred, pos_label=-1)

print(f"測試集性能:")
print(f"準確率: {test_accuracy:.4f}")
print(f"精確率: {test_precision:.4f}")
print(f"召回率: {test_recall:.4f}")
print(f"F1分數: {test_f1:.4f}")

# 記錄測試集性能指標
with open(f"{project_dir}/results/test_performance.txt", 'w') as f:
    f.write(f"模型: optimized_{best_model_type}\n")
    f.write(f"準確率: {test_accuracy:.4f}\n")
    f.write(f"精確率: {test_precision:.4f}\n")
    f.write(f"召回率: {test_recall:.4f}\n")
    f.write(f"F1分數: {test_f1:.4f}\n")

為模型 'mlp' 進行參數優化...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
最佳參數: {'alpha': 0.01, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
最佳交叉驗證F1分數: 0.9563
測試集性能:
準確率: 0.9625
精確率: 0.9963
召回率: 0.9283
F1分數: 0.9611


# 第四階段：特徵重要性分析
## 步驟 7：解釋模型預測與特徵重要性

In [ ]:
import shap
from sklearn.inspection import permutation_importance

# 載入最佳模型和處理後的數據
best_model = joblib.load(f"{project_dir}/models/optimized_{best_model_type}_model.pkl")
feature_set_name = 'anova_f'  # 使用與訓練相同的特徵集
data = pd.read_csv(f"{project_dir}/data/feature_set_{feature_set_name}.csv")
y = data['label']
X = data.drop('label', axis=1)

# 方法1: Built-in Feature Importance (適用於樹模型)
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8), facecolor='white')  # 設定背景為白色
    sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
    plt.title('Top 20 Feature Importance (Built-in)')
    plt.tight_layout()
    plt.savefig(f"{project_dir}/visualizations/feature_importance_built_in.png")
    plt.close()
    
    feature_importance.to_csv(f"{project_dir}/results/feature_importance_built_in.csv", index=False)

# 方法2: Permutation Importance (適用於所有模型)
print("Calculating Permutation Importance...")
try:
    result = permutation_importance(best_model, X_val, y_val, n_repeats=10, random_state=42, n_jobs=-1)
    permutation_importances = pd.DataFrame({
        'feature': X.columns,
        'importance': result.importances_mean
    }).sort_values('importance', ascending=False)

    plt.figure(figsize=(12, 8), facecolor='white')  # 設定背景為白色
    sns.barplot(x='importance', y='feature', data=permutation_importances.head(20))
    plt.title('Top 20 Feature Importance (Permutation)')
    plt.tight_layout()
    plt.savefig(f"{project_dir}/visualizations/permutation_importance.png")
    plt.close()

    permutation_importances.to_csv(f"{project_dir}/results/permutation_importance.csv", index=False)
except Exception as e:
    print(f"Error in calculating permutation importance: {e}")


# 方法3: SHAP值 (適用於所有模型)
print("計算SHAP值...")
try:
    # 創建SHAP解釋器
    if best_model_type in ['random_forest', 'gradient_boosting']:
        explainer = shap.TreeExplainer(best_model)
    else:
        # 對於非樹模型，使用KernelExplainer但取樣以提高效率
        X_sample = shap.sample(X_val, 100)
        explainer = shap.KernelExplainer(best_model.predict, X_sample)
        
    # 計算SHAP值，使用較少樣本以加快計算
    X_shap = X_val.iloc[:50] if hasattr(X_val, 'iloc') else X_val[:50]
    shap_values = explainer.shap_values(X_shap)
    
    # 檢查shap_values的結構並進行適當處理
    if isinstance(shap_values, list):
        # 對於分類問題，通常會返回每個類別的SHAP值列表
        print(f"SHAP值結構: 列表長度={len(shap_values)}")
        if len(shap_values) == 2:
            # 二分類問題，使用第二個類別(通常是正類)的SHAP值
            print("使用二分類問題的正類SHAP值")
            values_for_plot = shap_values[1]
            # 確保是一維數組
            if hasattr(values_for_plot, 'shape') and len(values_for_plot.shape) > 1:
                print(f"SHAP值形狀: {values_for_plot.shape}")
        else:
            # 多分類問題，使用所有類別SHAP值的絕對值平均
            print("使用多分類問題的SHAP值絕對值平均")
            values_for_plot = np.abs(np.array(shap_values)).mean(axis=0)
    else:
        # 對於回歸問題，直接使用SHAP值
        print("使用回歸問題的SHAP值")
        values_for_plot = shap_values
    
    # 確保數據是正確的形狀
    print(f"X_shap形狀: {X_shap.shape}")
    if hasattr(values_for_plot, 'shape'):
        print(f"values_for_plot形狀: {values_for_plot.shape}")
    
    # 嘗試繪製 SHAP 摘要圖
    try:
        # 繪製 Bar Summary Plot
        plt.figure(figsize=(12, 10), facecolor='white')  # 設定整張圖的背景為白色
        shap.summary_plot(values_for_plot, X_shap, plot_type='bar', show=False)
        plt.gca().set_facecolor('white')  # 設定座標區域背景為白色
        plt.tight_layout()
        plt.savefig(f"{project_dir}/visualizations/shap_summary_bar.png", facecolor='white')  # 確保輸出圖片背景為白色
        plt.close()
        
        # 繪製 Dot Summary Plot
        plt.figure(figsize=(12, 10), facecolor='white')  # 設定整張圖的背景為白色
        shap.summary_plot(values_for_plot, X_shap, show=False)
        plt.gca().set_facecolor('white')  # 設定座標區域背景為白色
        plt.tight_layout()
        plt.savefig(f"{project_dir}/visualizations/shap_summary_dot.png", facecolor='white')  # 確保輸出圖片背景為白色
        plt.close()

    except Exception as e:
        print(f"Error in SHAP summary plot: {e}")

    
    # 計算特徵重要性
    try:
        feature_names = list(X_shap.columns) if hasattr(X_shap, 'columns') else [f"feature_{i}" for i in range(X_shap.shape[1])]
        
        # 處理不同形狀的SHAP值
        if len(values_for_plot.shape) == 3:  # 可能是(樣本數, 時間步長, 特徵數)
            importance_values = np.abs(values_for_plot).mean(axis=(0, 1))
        elif len(values_for_plot.shape) == 2:  # 標準形狀(樣本數, 特徵數)
            importance_values = np.abs(values_for_plot).mean(axis=0)
        else:
            raise ValueError(f"無法處理形狀為{values_for_plot.shape}的SHAP值")
        
        # 確保特徵名稱和重要性值的長度一致
        if len(feature_names) != len(importance_values):
            print(f"警告: 特徵名稱數量({len(feature_names)})與重要性值數量({len(importance_values)})不匹配")
            # 裁剪較長的列表以匹配較短的列表
            min_len = min(len(feature_names), len(importance_values))
            feature_names = feature_names[:min_len]
            importance_values = importance_values[:min_len]
        
        shap_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': importance_values
        }).sort_values('importance', ascending=False)
        
        shap_importance.to_csv(f"{project_dir}/results/shap_importance.csv", index=False)
        
        # 繪製 SHAP 依賴圖
        top_features = shap_importance['feature'].head(5).tolist()
        for i, feature in enumerate(top_features):
            plt.figure(figsize=(10, 7), facecolor='white')  # 設定整張圖的背景顏色
            feature_idx = list(feature_names).index(feature)
            
            # 繪製 SHAP 依賴圖
            shap.dependence_plot(feature_idx, values_for_plot, X_shap, show=False)
            
            # 設定座標區域的背景顏色為白色
            plt.gca().set_facecolor('white')  
            
            # 設定標題 (改為英文)
            plt.title(f'SHAP Dependence Plot - {feature}')
            
            plt.tight_layout()
            plt.savefig(f"{project_dir}/visualizations/shap_dependence_{i}_{feature}.png", facecolor='white')  # 確保輸出圖片背景為白色
            plt.close()

    except Exception as e:
        print(f"SHAP特徵重要性計算錯誤: {e}")
except Exception as e:
    print(f"SHAP解釋器創建錯誤: {e}")

Calculating Permutation Importance...
計算SHAP值...


  0%|          | 0/50 [00:00<?, ?it/s]

使用回歸問題的SHAP值
X_shap形狀: (50, 50)
values_for_plot形狀: (50, 50)


<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

<Figure size 720x504 with 0 Axes>

# 第五階段：實際應用
## 步驟 8：生成模擬測試資料的程式

In [11]:
import pandas as pd
import numpy as np
import os

# 假設選擇的特徵模式是 'anova_f'，包含50個特徵
np.random.seed(42)  # 設定隨機種子以保證可重現性

# 創建目錄（如果不存在）
if not os.path.exists('sample_data'):
    os.makedirs('sample_data')

# 從原始數據了解特徵的分佈情況
# 這裡使用假設的數據範圍，實際使用時應從原始數據中獲取
num_samples = 20  # 模擬數據的樣本數
num_features = 50  # 與我們訓練模型使用的特徵數相匹配

# 模擬特徵的名稱，與實際訓練模型的特徵名稱相匹配
# 如果模型訓練時使用的是數字索引，應使用相同格式
feature_names = [f'Feature_{i}' for i in range(num_features)]

# 創建模擬數據
# 這裡的值範圍是任意設定的，應該調整為與實際數據相似的範圍
simulated_data = np.random.normal(0, 1, size=(num_samples, num_features))

# 創建DataFrame
test_df = pd.DataFrame(simulated_data, columns=feature_names)

# 保存模擬測試數據
test_df.to_csv('sample_data/test_data.csv', index=False)

print(f"已生成包含 {num_samples} 筆資料和 {num_features} 個特徵的測試檔案: sample_data/test_data.csv")

已生成包含 20 筆資料和 50 個特徵的測試檔案: sample_data/test_data.csv


## 步驟 9：進行預測

In [12]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import StandardScaler

# 首先生成測試數據
def generate_test_data(output_file='sample_data/test_data.csv', num_samples=20):
    """生成模擬測試數據"""
    # 設定隨機種子以保證可重現性
    np.random.seed(42)
    
    # 創建目錄（如果不存在）
    if not os.path.exists('sample_data'):
        os.makedirs('sample_data')
    
    # 特徵數
    num_features = 50
    
    # 特徵名稱
    feature_names = [f'Feature_{i}' for i in range(num_features)]
    
    # 創建模擬數據
    simulated_data = np.random.normal(0, 1, size=(num_samples, num_features))
    
    # 創建DataFrame
    test_df = pd.DataFrame(simulated_data, columns=feature_names)
    
    # 保存模擬測試數據
    test_df.to_csv(output_file, index=False)
    
    print(f"已生成包含 {num_samples} 筆資料和 {num_features} 個特徵的測試檔案: {output_file}")
    return test_df

# 預測函數
def predict_failures(input_data=None, input_file=None, output_file='results/predictions.csv', model_path='semiconductor_maintenance/models/optimized_mlp_model.pkl'):
    """
    使用訓練好的模型對輸入的數據進行故障預測
    
    參數:
    input_data (DataFrame): 輸入特徵數據的DataFrame
    input_file (str): 或者，輸入CSV檔案路徑，包含特徵數據
    output_file (str): 輸出CSV檔案路徑，將存儲預測結果
    model_path (str): 模型檔案路徑
    """
    try:
        # 確保 results 目錄存在
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        print(f"正在載入模型: {model_path}")
        if not os.path.exists(model_path):
            print(f"錯誤: 找不到模型檔案 {model_path}")
            print("注意: 在此演示中，我們將模擬模型預測，實際使用時請確保模型檔案存在")
            # 我們將模擬模型預測
            class DummyModel:
                def predict(self, X):
                    # 隨機生成預測結果，約80%是故障(-1)，20%是正常(1)
                    return np.random.choice([-1, 1], size=X.shape[0], p=[0.8, 0.2])
                
                def predict_proba(self, X):
                    # 隨機生成預測機率
                    probs = np.random.rand(X.shape[0], 2)
                    probs = probs / probs.sum(axis=1, keepdims=True)  # 歸一化以確保總和為1
                    return probs
                
                @property
                def classes_(self):
                    return np.array([-1, 1])
            
            model = DummyModel()
        else:
            # 載入真實模型
            model = joblib.load(model_path)
        
        # 如果提供了 input_file 而非 input_data，則從檔案讀取數據
        if input_data is None and input_file is not None:
            print(f"正在讀取輸入檔案: {input_file}")
            input_data = pd.read_csv(input_file)
        
        if input_data is None:
            print("錯誤: 請提供輸入數據或輸入檔案路徑")
            return False
        
        # 保存原始索引以便於追蹤結果
        original_index = input_data.index
        
        print(f"輸入數據形狀: {input_data.shape}")
        
        # 檢查特徵數量是否符合預期
        expected_features = 50  # 假設模型使用的是50個 ANOVA 選出的特徵
        
        if input_data.shape[1] != expected_features:
            print(f"警告: 輸入數據的特徵數 ({input_data.shape[1]}) 與模型預期的特徵數 ({expected_features}) 不符")
            print("請確保輸入數據包含正確的特徵，按照與模型訓練時相同的順序排列")
        
        # 標準化數據
        print("正在預處理數據...")
        scaler = StandardScaler()
        input_data_scaled = scaler.fit_transform(input_data)
        
        # 進行預測
        print("正在進行預測...")
        predictions = model.predict(input_data_scaled)
        
        # 轉換預測結果為易於理解的格式
        # 假設 -1 表示故障，1 表示正常
        prediction_labels = ["故障" if pred == -1 else "正常" for pred in predictions]
        
        # 如果模型支援機率預測，也輸出故障的機率
        if hasattr(model, "predict_proba"):
            try:
                proba = model.predict_proba(input_data_scaled)
                # 確認模型的類別順序，這裡假設第一個是-1 (故障)，第二個是1 (正常)
                if len(model.classes_) == 2:
                    if model.classes_[0] == -1:
                        failure_proba = proba[:, 0]
                    else:
                        failure_proba = proba[:, 1]
                else:
                    # 如果模型類別順序不明確，只輸出預測標籤
                    failure_proba = None
            except:
                failure_proba = None
        else:
            failure_proba = None
        
        # 創建結果 DataFrame
        results = pd.DataFrame({'原始索引': original_index, '預測結果': prediction_labels})
        
        # 如果有故障機率，也加入結果
        if failure_proba is not None:
            results['故障機率'] = failure_proba
        
        # 保存結果
        results.to_csv(output_file, index=False)
        print(f"預測完成，結果已儲存至: {output_file}")
        print(f"共預測 {len(predictions)} 筆數據，其中故障 {sum(predictions == -1)} 筆，正常 {sum(predictions == 1)} 筆")
        
        # 在 Jupyter 中顯示結果
        display(results)
        
        return results
    
    except Exception as e:
        print(f"發生錯誤: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# 範例執行
# 1. 生成測試數據
print("步驟1: 生成測試數據")
test_data = generate_test_data()

# 2. 使用生成的測試數據進行預測
print("\n步驟2: 使用測試數據進行預測")
# 注意這裡使用DataFrame參數，而不是檔案路徑
prediction_results = predict_failures(input_data=test_data)

步驟1: 生成測試數據
已生成包含 20 筆資料和 50 個特徵的測試檔案: sample_data/test_data.csv

步驟2: 使用測試數據進行預測
正在載入模型: semiconductor_maintenance/models/optimized_mlp_model.pkl
輸入數據形狀: (20, 50)
正在預處理數據...
正在進行預測...
預測完成，結果已儲存至: results/predictions.csv
共預測 20 筆數據，其中故障 16 筆，正常 4 筆


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,原始索引,預測結果,故障機率
0,0,故障,0.985654
1,1,正常,0.070757
2,2,故障,0.948307
3,3,故障,0.996609
4,4,正常,0.046959
5,5,故障,0.999997
6,6,故障,0.999941
7,7,故障,0.999911
8,8,故障,1.000000
9,9,故障,1.000000
